In [ ]:
%%bash
#pip install TextBlob
#conda install gensim

In [1]:
import pandas as pd
import numpy as np
from gensim import corpora, models, similarities
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
from gensim import corpora, models, similarities
import slugify as sl
import pickle
import re
import RAKE

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [3]:
movieplots = pd.read_csv(dropbox + 'movieplots.csv')

In [4]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [5]:
#this is because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

movieplots = movieplots.apply(clean, axis = 1)

In [6]:
dftouse = movieplots[movieplots.year > 2014]

In [7]:
print movieplots.shape
print dftouse.shape
dftouse.head()

(259028, 5)
(16943, 5)


,title,source,year,plots,type
2,#1 at the Apocalypse Box Office (2015),imdb,2015,"[Jules is, self declared, the most useless per...",user plot
9,#50Fathers (2015),imdb,2015,[#50Fathers is an American Dramatic Comedy. Do...,user plot
15,#BeRobin the Movie (2015),imdb,2015,[A documentary about Margaret Cho's homeless o...,user plot
16,#Beings (2015),imdb,2015,[Beings is the second feature film of Stefanes...,user plot
20,#Disneyland60 (2015),imdb,2015,"[Kate, a 15 year old college student documents...",user plot


In [8]:
def slugify (text):
    return sl.slugify(text)

In [9]:
def sentences(text):
    blob = tb.TextBlob(text.decode('unicode-escape', 'ignore'))
    return blob.sentences

In [10]:
def stem(word):
    return PorterStemmer().stem(word)

In [11]:
num = re.compile('\d')
def contains_number(word):
    return bool(num.search(word))

In [12]:
Rake = RAKE.Rake('/Users/mr/Devel/Harvard/CS109/Project/moviemeta/data/stoplists/FoxStoplist.txt')
def keywords(text):
    keywords = Rake.run(text)
    return ' '.join([tup[0] for tup in keywords])

In [59]:
def process(sents, movie_id=None, stop=False, postags=None ):
    if stop:
        stopwords = set(sw.words('english'))
    doc_sents  = []
    doc_words  = []
    for sent in sents:
        if postags:
            words = sent.tags
        else:
            words = sent.words
        words_processed = []
        for word in words:
            if postags:
                if word[1] not in postags:
                    continue
                else:
                    word = word[0]
            if stop and word in stopwords:
                    continue
            if contains_number(word):
                    continue
            word = stem(slugify(word))
            if len(word) < 2:
                continue
            words_processed.append(word)
        doc_sents.append(models.doc2vec.LabeledSentence(words_processed,[movie_id]))
        doc_words += words_processed 
    return (doc_sents, doc_words)

In [14]:
#do the processing for topic models
documents_tm = dftouse['plots'].apply(lambda x : process(sentences(keywords(' '.join(x))))[1])

In [60]:
#do the processing for doc2vec
def process_d2v(row):
    movie = row[0]
    plots = ' '.join(row[1])
    return  process(sentences(plots), movie_id=movie, stop=True)[0]
documents_d2v = dftouse.head(100)[['title','plots']].apply(process_d2v, axis=1)

In [64]:
#models.Doc2Vec(documents_d2v)
sents=[]
for doc in documents_d2v:
    for sent in doc:
        sents.append(sent)
d2v = models.Doc2Vec(sents, dbow_words=1)

In [68]:
#print sents[0][1]

d2v.docvecs.most_similar(sents[0][1])

[('10 Milliarden (2015)', 0.17417208850383759),
 ('100 Miles to Lordsburg (2015)', 0.13997596502304077),
 ('12 Rounds 3: Lockdown (2015)', 0.12714487314224243),
 ('1915 (2015)', 0.10568301379680634),
 ('19-2 on Bravo: The Dark Side (2015) (V)', 0.09917952865362167),
 ('11:55 Holyoke (2015)', 0.09635968506336212),
 ('1 Building and 40 People Dancing (2015)', 0.09364811331033707),
 ('1945. Kalniskes- luzio taskas/1945. Kalniskes- a breakpoint (2016)',
  0.09166409075260162),
 ('10,000 Miles (2016)', 0.08629010617733002),
 ('0-1 (2015)', 0.08533351123332977)]

In [20]:
print dftouse['plots'][2]
print documents[2]

['Jules is, self declared, the most useless person in the post apocalyptic world, until he finds an old film camera and determines to make the greatest movie in the new world... the only movie in the new world. But his first day filming is proving to be much more difficult than he imagined. Who knew that making a movie after the end of the world would be so hard?']
[u'post', u'apocalypt', u'world', u'day', u'film', u'film', u'camera', u'useless', u'person', u'self', u'declar', u'world', u'determin', u'movi', u'prove', u'jule', u'imagin', u'hard', u'difficult']


In [16]:
#save the correspondin movies
movies = dftouse['title'].values
with open(dropbox +'imdb_movies_since_2014.pickle', 'wb') as f:
    np.save(f,movies)

### Example

In [ ]:
doc = """St. Johns, Newfoundland is a city with a deep musical spirit and few bands embody that spirit better that The Once. Named for a unique Newfoundland phrase that means "imminently," now is indeed their time. Live at the Stagehouse is an hour of interviews and performances by this amazing trio, featuring songs recorded live off the floor in late 2013 at the picturesque Stagehouse Recording Studio in St. Philip's. The Once embraces a different vision of Newfoundland music. Their sounds do not come from the noisy pubs and dockside taverns that fuel so much of the Island's energy. Instead, their music comes from a quieter and more thoughtful place. Hope and tragedy are intertwined, whether they are singing an old lament from World War I, original songs that speak of love defeated, or tasteful songs from the artists whose music inspires them. The Once: Live at the Stagehouse features some of their best-loved tunes and some never heard before. It is a unique opportunity to see one of Canada's fastest rising bands in the place that inspires them and feeds their soul."""

In [ ]:
%time
test = process(sentences(doc))
%time
testpostags = process(sentences(doc),postags=['FW', 'JJ','JJR','JJS','NN','NNS','NNP', 'NNPS','VB','VBD','VBG','VBN','VBP','VBZ'])
%time
testkeywords = process(sentences(keywords(doc)))

In [38]:
print testkeywords
print testpostags

-1

In [ ]:
#process with spark
'''import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)
plots_sdf = sqlsc.createDataFrame(dftouse)
plots = (plots_sdf[['title','plots']]
    .map(lambda x : (' '.join(x[1])))
)
docs = (plots
    .map(sentences)
    .map(process)          
).cache()
docs_sents = docs.map(lambda x : x[0]).collect()
docs_words = docs.map(lambda x : x[1]).collect()
'''